In [1]:
# the imports
from pymongo import MongoClient
from pprint import pprint
import pandas as pd
from venmo_helper import print_schema 

# the configuration
## print full text in panda
pd.set_option('display.max_colwidth', -1) 

# the database connection
client = MongoClient('localhost:27017')
db = client.venmo
venmo = db.venmo

transactions = venmo.aggregate([{ '$sample' : {'size' : 1}}])
for transaction in transactions:
    print_schema(transaction)

{
	_id, date_updated, transfer, id, note, audience, 
	date_created, type, authorization 
	app {
		id, image_url, description, name, site_url }
	comments {
		count, data }
	likes {
		count, data }
	mentions {
		count, data }
	payment {
		status, merchant_split_purchase, date_completed, amount, note, audience, 
		action, date_created, date_reminded, id, date_authorized 
		target {
			merchant, redeemable_target, phone, type, email 
			user {
				username, about, last_name, display_name, friends_count, is_group, 
				is_active, trust_request, email, phone, profile_picture_url, first_name, 
				friend_status, is_blocked, id, identity, date_joined }}
		actor {
			username, about, last_name, display_name, friends_count, is_group, 
			is_active, trust_request, email, phone, profile_picture_url, first_name, 
			friend_status, is_blocked, id, identity, date_joined }}}

In [5]:
## aggregate by different types of apps used
transactions =  venmo.aggregate( [{ '$group' : { '_id' : {'description': '$app.description', 'url' : '$app.site_url'} , 'count' : { '$sum' : 1}}}])
df = pd.DataFrame((item['_id']['description'], item['_id']['url'], item['count']) for item in transactions)
df.columns=['Description', 'URL', 'Count']
df.sort_values(by=['Count'], ascending=False).style.hide_index()

Description,URL,Count
Venmo for iPhone,None,6156264
Venmo for Android,None,884654
"Share bills and IOUs. Make sure everyone gets paid back. Totally free for web, iPhone, and Android.",https://secure.splitwise.com/,25621
Venmo.com,None,9271
Payouts API,None,484
Makes splitting restaurant/bar bills easy; facilitates payments between people in the group.,http://www.tabapp.co/,201
Venmo Developer,None,61
Used for Bottle Rocket Utility,None,6
Automation,None,6
Transfer money to phone contacts in a swipe,None,6


In [6]:
## aggregate by email
transactions =  venmo.aggregate( [{ '$group' : { '_id' : {'email': '$payment.actor.email'} , 'count' : { '$sum' : 1}}}])
df = pd.DataFrame((item['_id']['email'], item['count']) for item in transactions)
df.columns=['Email', 'Count']
df.sort_values(by=['Count'], ascending=False).style.hide_index()

Email,Count
None,7076585


In [7]:
## aggregate by payment amount
transactions =  venmo.aggregate( [{ '$group' : { '_id' : {'amount': '$payment.amount'} , 'count' : { '$sum' : 1}}}])
df = pd.DataFrame((item['_id']['amount'], item['count']) for item in transactions)
df.columns=['Amount', 'Count']
df.sort_values(by=['Amount'], ascending=False).style.hide_index()

Amount,Count
None,7076585


In [2]:
df = pd.read_csv('users.csv',
                dtype={'is_blocked':object, 'is_group' : object, 'is_active': object},
                 usecols=['username', 'date_joined']                
                )

In [3]:
df.username.unique().size

7239701